# ZuCo 1.0 & 2.0 EDA (Exploratory Data Analysis)

This notebook explores the ZuCo datasets for EEG-to-Text research.

**Datasets:**
- **ZuCo 1.0**: 12 subjects (Z prefix), 3 tasks
- **ZuCo 2.0**: 18 subjects (Y prefix), 1 task

In [1]:
# Install dependencies
!pip install -q matplotlib seaborn pandas numpy scipy

In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("Libraries loaded!")

Libraries loaded!


## 1. Load Data

In [7]:
# Update these paths to match your data location
ZUCO1_PATHS = {
    'task1-SR': 'C:\\MSc Files\\MSc Project\\E2T-w-VJEPA\\e2t-w-jepa-pretraining\\dataset\\ZuCo\\task1-SR\\pickle\\task1-SR-dataset-spectro.pickle',
    'task2-NR': 'C:\\MSc Files\\MSc Project\\E2T-w-VJEPA\\e2t-w-jepa-pretraining\\dataset\\ZuCo\\task2-NR\\pickle\\task2-NR-dataset-spectro.pickle',
    'task3-TSR': 'C:\\MSc Files\\MSc Project\\E2T-w-VJEPA\\e2t-w-jepa-pretraining\\dataset\\ZuCo\\task3-TSR\\pickle\\task3-TSR-dataset-spectro.pickle',
}

ZUCO2_PATHS = {
    'task2-NR-2.0': 'C:\\MSc Files\\MSc Project\\E2T-w-VJEPA\\e2t-w-jepa-pretraining\\dataset\\ZuCo\\task2-NR-2.0\\pickle\\task2-NR-2.0-dataset-spectro.pickle',
}

def load_pickle(path):
    """Load pickle file."""
    if not Path(path).exists():
        print(f"⚠️ File not found: {path}")
        return None
    print(f"Loading {Path(path).name}...")
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

In [10]:
# Load ZuCo 1.0
zuco1_data = {}
for task_name, path in ZUCO1_PATHS.items():
    data = load_pickle(path)
    if data:
        zuco1_data[task_name] = data
        print(f"  ✓ {task_name}: {len(data)} subjects")

# Load ZuCo 2.0
zuco2_data = {}
for task_name, path in ZUCO2_PATHS.items():
    data = load_pickle(path)
    if data:
        zuco2_data[task_name] = data
        print(f"  ✓ {task_name}: {len(data)} subjects")

Loading task1-SR-dataset-spectro.pickle...
  ✓ task1-SR: 12 subjects
Loading task2-NR-dataset-spectro.pickle...
  ✓ task2-NR: 12 subjects
Loading task3-TSR-dataset-spectro.pickle...
  ✓ task3-TSR: 12 subjects
Loading task2-NR-2.0-dataset-spectro.pickle...


KeyboardInterrupt: 

## 2. Dataset Overview

In [ ]:
def get_dataset_stats(data, dataset_name):
    """Extract statistics from pickle data."""
    stats = {
        'dataset': dataset_name,
        'subjects': list(data.keys()),
        'n_subjects': len(data),
        'total_sentences': 0,
        'sentences_per_subject': [],
        'text_lengths': [],
        'eeg_time_lengths': [],
    }
    
    for subject_id, sentences in data.items():
        n_sentences = len(sentences)
        stats['total_sentences'] += n_sentences
        stats['sentences_per_subject'].append(n_sentences)
        
        for sent in sentences:
            if sent is None:
                continue
            # Text length
            text = sent.get('content', '')
            stats['text_lengths'].append(len(text.split()))
            
            # EEG time length
            eeg_data = sent.get('sentence_level_EEG', {})
            if 'rawData' in eeg_data:
                raw = np.array(eeg_data['rawData'])
                if raw.ndim == 2:
                    stats['eeg_time_lengths'].append(raw.shape[1] if raw.shape[0] == 105 else raw.shape[0])
    
    return stats

# Collect stats
all_stats = []
for task_name, data in zuco1_data.items():
    all_stats.append(get_dataset_stats(data, f"ZuCo1-{task_name}"))
for task_name, data in zuco2_data.items():
    all_stats.append(get_dataset_stats(data, f"ZuCo2-{task_name}"))

# Display summary
summary_df = pd.DataFrame([{
    'Dataset': s['dataset'],
    'Subjects': s['n_subjects'],
    'Total Sentences': s['total_sentences'],
    'Avg Sentences/Subject': s['total_sentences'] / s['n_subjects'] if s['n_subjects'] > 0 else 0,
    'Avg Words/Sentence': np.mean(s['text_lengths']) if s['text_lengths'] else 0,
    'Avg EEG Time (samples)': np.mean(s['eeg_time_lengths']) if s['eeg_time_lengths'] else 0,
} for s in all_stats])

print("\n" + "="*80)
print("DATASET OVERVIEW")
print("="*80)
display(summary_df.round(2))

## 3. Subject Analysis

In [ ]:
# ZuCo 1.0 subjects
zuco1_subjects = set()
for data in zuco1_data.values():
    zuco1_subjects.update(data.keys())

# ZuCo 2.0 subjects
zuco2_subjects = set()
for data in zuco2_data.values():
    zuco2_subjects.update(data.keys())

print(f"ZuCo 1.0 Subjects ({len(zuco1_subjects)}): {sorted(zuco1_subjects)}")
print(f"ZuCo 2.0 Subjects ({len(zuco2_subjects)}): {sorted(zuco2_subjects)}")
print(f"\nTotal unique subjects: {len(zuco1_subjects | zuco2_subjects)}")

In [ ]:
# Sentences per subject per task
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ZuCo 1.0
ax = axes[0]
for task_name, data in zuco1_data.items():
    subjects = list(data.keys())
    counts = [len(data[s]) for s in subjects]
    ax.bar([f"{s}" for s in subjects], counts, alpha=0.7, label=task_name)
ax.set_xlabel('Subject')
ax.set_ylabel('Number of Sentences')
ax.set_title('ZuCo 1.0: Sentences per Subject')
ax.legend()
ax.tick_params(axis='x', rotation=45)

# ZuCo 2.0
ax = axes[1]
for task_name, data in zuco2_data.items():
    subjects = list(data.keys())
    counts = [len(data[s]) for s in subjects]
    ax.bar(subjects, counts, alpha=0.7, label=task_name)
ax.set_xlabel('Subject')
ax.set_ylabel('Number of Sentences')
ax.set_title('ZuCo 2.0: Sentences per Subject')
ax.legend()
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Text Analysis

In [ ]:
def collect_texts(data):
    """Collect all texts from data."""
    texts = []
    for sentences in data.values():
        for sent in sentences:
            if sent and 'content' in sent:
                texts.append(sent['content'])
    return texts

# Collect all texts
all_texts = {'ZuCo 1.0': [], 'ZuCo 2.0': []}
for data in zuco1_data.values():
    all_texts['ZuCo 1.0'].extend(collect_texts(data))
for data in zuco2_data.values():
    all_texts['ZuCo 2.0'].extend(collect_texts(data))

print(f"ZuCo 1.0 sentences: {len(all_texts['ZuCo 1.0'])}")
print(f"ZuCo 2.0 sentences: {len(all_texts['ZuCo 2.0'])}")

In [ ]:
# Text length distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, (name, texts) in enumerate(all_texts.items()):
    ax = axes[idx]
    word_counts = [len(t.split()) for t in texts]
    char_counts = [len(t) for t in texts]
    
    ax.hist(word_counts, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
    ax.axvline(np.mean(word_counts), color='red', linestyle='--', label=f'Mean: {np.mean(word_counts):.1f}')
    ax.axvline(np.median(word_counts), color='orange', linestyle='--', label=f'Median: {np.median(word_counts):.1f}')
    ax.set_xlabel('Words per Sentence')
    ax.set_ylabel('Frequency')
    ax.set_title(f'{name}: Text Length Distribution')
    ax.legend()

plt.tight_layout()
plt.show()

# Print stats
for name, texts in all_texts.items():
    word_counts = [len(t.split()) for t in texts]
    print(f"\n{name} Text Stats:")
    print(f"  Min words: {min(word_counts)}")
    print(f"  Max words: {max(word_counts)}")
    print(f"  Mean words: {np.mean(word_counts):.2f}")
    print(f"  Std words: {np.std(word_counts):.2f}")

In [ ]:
# Sample sentences
print("\n" + "="*80)
print("SAMPLE SENTENCES")
print("="*80)

for name, texts in all_texts.items():
    print(f"\n{name}:")
    for i, text in enumerate(texts[:5]):
        print(f"  [{i+1}] {text[:100]}{'...' if len(text) > 100 else ''}")

## 5. EEG Data Analysis

In [ ]:
def get_eeg_sample(data, n=1):
    """Get sample EEG data from dataset."""
    samples = []
    for sentences in data.values():
        for sent in sentences:
            if sent is None:
                continue
            eeg_data = sent.get('sentence_level_EEG', {})
            if 'rawData' in eeg_data:
                raw = np.array(eeg_data['rawData'])
                if raw.ndim == 2 and raw.shape[0] == 105:
                    samples.append({
                        'raw': raw,
                        'text': sent.get('content', ''),
                    })
                    if len(samples) >= n:
                        return samples
    return samples

# Get sample from first available dataset
sample_data = None
if zuco1_data:
    first_task = list(zuco1_data.keys())[0]
    sample_data = get_eeg_sample(zuco1_data[first_task], n=3)
elif zuco2_data:
    first_task = list(zuco2_data.keys())[0]
    sample_data = get_eeg_sample(zuco2_data[first_task], n=3)

if sample_data:
    print(f"Sample EEG shape: {sample_data[0]['raw'].shape}")
    print(f"  Channels: {sample_data[0]['raw'].shape[0]}")
    print(f"  Time samples: {sample_data[0]['raw'].shape[1]}")

In [ ]:
# Visualize EEG data
if sample_data:
    fig, axes = plt.subplots(len(sample_data), 1, figsize=(14, 4*len(sample_data)))
    if len(sample_data) == 1:
        axes = [axes]
    
    for idx, sample in enumerate(sample_data):
        ax = axes[idx]
        eeg = sample['raw']
        text = sample['text'][:50] + '...' if len(sample['text']) > 50 else sample['text']
        
        # Plot subset of channels
        n_channels_to_plot = 10
        for ch in range(0, 105, 105 // n_channels_to_plot):
            ax.plot(eeg[ch, :500], alpha=0.7, linewidth=0.5)
        
        ax.set_xlabel('Time (samples)')
        ax.set_ylabel('Amplitude')
        ax.set_title(f'EEG Signal - "{text}"')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# EEG time length distribution
eeg_lengths = {'ZuCo 1.0': [], 'ZuCo 2.0': []}

for data in zuco1_data.values():
    for sentences in data.values():
        for sent in sentences:
            if sent is None:
                continue
            eeg_data = sent.get('sentence_level_EEG', {})
            if 'rawData' in eeg_data:
                raw = np.array(eeg_data['rawData'])
                if raw.ndim == 2 and raw.shape[0] == 105:
                    eeg_lengths['ZuCo 1.0'].append(raw.shape[1])

for data in zuco2_data.values():
    for sentences in data.values():
        for sent in sentences:
            if sent is None:
                continue
            eeg_data = sent.get('sentence_level_EEG', {})
            if 'rawData' in eeg_data:
                raw = np.array(eeg_data['rawData'])
                if raw.ndim == 2 and raw.shape[0] == 105:
                    eeg_lengths['ZuCo 2.0'].append(raw.shape[1])

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for idx, (name, lengths) in enumerate(eeg_lengths.items()):
    if not lengths:
        continue
    ax = axes[idx]
    ax.hist(lengths, bins=50, alpha=0.7, color='seagreen', edgecolor='black')
    ax.axvline(np.mean(lengths), color='red', linestyle='--', label=f'Mean: {np.mean(lengths):.0f}')
    ax.axvline(500, color='purple', linestyle=':', label='Max (500)')
    ax.set_xlabel('Time Samples')
    ax.set_ylabel('Frequency')
    ax.set_title(f'{name}: EEG Sequence Length')
    ax.legend()

plt.tight_layout()
plt.show()

# Print stats
for name, lengths in eeg_lengths.items():
    if lengths:
        print(f"\n{name} EEG Length Stats:")
        print(f"  Min: {min(lengths)} samples")
        print(f"  Max: {max(lengths)} samples")
        print(f"  Mean: {np.mean(lengths):.0f} samples")
        print(f"  % < 500: {100 * sum(1 for l in lengths if l < 500) / len(lengths):.1f}%")

## 6. Channel Analysis

In [ ]:
# Brain region mapping
BRAIN_REGIONS = {
    'Prefrontal L': list(range(0, 11)),
    'Prefrontal R': list(range(11, 22)),
    'Frontal L': list(range(22, 33)),
    'Frontal R': list(range(33, 44)),
    'Central L': list(range(44, 55)),
    'Central R': list(range(55, 66)),
    'Temporal L': list(range(66, 77)),
    'Temporal R': list(range(77, 88)),
    'Parietal-Occipital L': list(range(88, 97)),
    'Parietal-Occipital R': list(range(97, 105)),
}

print("Brain Region Channel Mapping:")
for region, channels in BRAIN_REGIONS.items():
    print(f"  {region}: {len(channels)} channels ({channels[0]}-{channels[-1]})")

In [ ]:
# Analyze channel statistics from samples
if sample_data:
    eeg = sample_data[0]['raw']
    
    # Per-channel statistics
    channel_means = np.mean(eeg, axis=1)
    channel_stds = np.std(eeg, axis=1)
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # Mean per channel
    ax = axes[0]
    colors = []
    for ch in range(105):
        for region, channels in BRAIN_REGIONS.items():
            if ch in channels:
                colors.append(list(BRAIN_REGIONS.keys()).index(region))
                break
    ax.bar(range(105), channel_means, color=plt.cm.tab10(np.array(colors) % 10), alpha=0.7)
    ax.set_xlabel('Channel')
    ax.set_ylabel('Mean Amplitude')
    ax.set_title('Mean Amplitude per Channel')
    
    # Std per channel
    ax = axes[1]
    ax.bar(range(105), channel_stds, color=plt.cm.tab10(np.array(colors) % 10), alpha=0.7)
    ax.set_xlabel('Channel')
    ax.set_ylabel('Std Amplitude')
    ax.set_title('Standard Deviation per Channel')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Per-region average power
if sample_data:
    eeg = sample_data[0]['raw']
    
    region_powers = []
    region_names = []
    for region, channels in BRAIN_REGIONS.items():
        power = np.mean(np.abs(eeg[channels, :]))
        region_powers.append(power)
        region_names.append(region)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    bars = ax.barh(region_names, region_powers, color=plt.cm.viridis(np.linspace(0.2, 0.8, len(region_names))))
    ax.set_xlabel('Average Power')
    ax.set_title('Average EEG Power by Brain Region')
    
    plt.tight_layout()
    plt.show()

## 7. Processed Data Analysis (if available)

In [ ]:
import torch

# Check for processed data
processed_dir = Path('./data/processed')
if processed_dir.exists():
    print("Processed data files:")
    for f in processed_dir.glob('*.pt'):
        data = torch.load(f, weights_only=False)
        if 'eeg' in data:
            print(f"  {f.name}: {data['eeg'].shape[0]} samples, EEG shape: {data['eeg'].shape[1:]}")
        else:
            print(f"  {f.name}: {list(data.keys())}")
else:
    print("No processed data found. Run prepare_pytorch_data.py first.")

In [ ]:
# Load and visualize processed data
if processed_dir.exists() and (processed_dir / 'train_data.pt').exists():
    train_data = torch.load(processed_dir / 'train_data.pt', weights_only=False)
    val_data = torch.load(processed_dir / 'val_data.pt', weights_only=False)
    test_data = torch.load(processed_dir / 'test_data.pt', weights_only=False)
    
    print("\nProcessed Dataset Splits:")
    print(f"  Train: {train_data['eeg'].shape[0]} samples")
    print(f"  Val: {val_data['eeg'].shape[0]} samples")
    print(f"  Test: {test_data['eeg'].shape[0]} samples")
    print(f"  Total: {train_data['eeg'].shape[0] + val_data['eeg'].shape[0] + test_data['eeg'].shape[0]} samples")
    print(f"  EEG shape: {train_data['eeg'].shape[1:]}")
    
    # Subject distribution
    train_subjects = Counter(train_data['subjects'])
    val_subjects = Counter(val_data['subjects'])
    test_subjects = Counter(test_data['subjects'])
    
    print(f"\nTrain subjects: {list(train_subjects.keys())}")
    print(f"Val subjects: {list(val_subjects.keys())}")
    print(f"Test subjects: {list(test_subjects.keys())}")

## 8. Summary

In [ ]:
print("\n" + "="*80)
print("SUMMARY")
print("="*80)

print("\n📊 Dataset Statistics:")
total_sentences = sum(len(t) for t in all_texts.values())
print(f"  Total sentences: {total_sentences:,}")
print(f"  Total subjects: {len(zuco1_subjects | zuco2_subjects)}")
print(f"  ZuCo 1.0 subjects: {len(zuco1_subjects)}")
print(f"  ZuCo 2.0 subjects: {len(zuco2_subjects)}")

print("\n📈 EEG Data:")
print(f"  Channels: 105")
print(f"  Time samples (spectro): variable, padded/cropped to 500")
print(f"  Brain regions: 10 (5 left, 5 right)")

print("\n📝 Text Statistics:")
all_word_counts = [len(t.split()) for texts in all_texts.values() for t in texts]
print(f"  Avg words/sentence: {np.mean(all_word_counts):.1f}")
print(f"  Max words: {max(all_word_counts)}")
print(f"  Min words: {min(all_word_counts)}")

print("\n✅ EDA Complete!")